In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import os
import numpy as np
import pandas as pd
import time
import torch
import pickle
import shap
import tensorboard


from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
from functools import partial
import shutil
from tqdm.auto import tqdm

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

from codes.models.data_form.DataForm import DataTransfo_1SNP
from codes.models.metrics import calculate_roc_auc

import featurewiz as gwiz

import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

from codes.models.Decision_tree.utils import get_indice, get_name

In [ ]:
### framework constants:
model_type = 'decision_tree'
model_version = 'gradient_boosting'
test_name = '1_test_train_transfo_V1'
tryout = True # True if we are ding a tryout, False otherwise 
### data constants:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Paul' # Paul, Abby
ld = 'no'
rollup_depth = 4
binary_classes = True #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = False
save_data = True
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = False
list_env_features = ['age', 'sex']
list_pheno_ids = None #list(np.load(f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/phewas/list_associations_snps/{SNP}_paul.npy'))

### data format

batch_size = 20
data_share = 1

##### model constants


##### training constants
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=remove_none,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share,
                         list_pheno_ids=list_pheno_ids,
                         binary_classes=binary_classes, 
                         ld = ld)
#patient_list = dataT.get_patientlist()

In [ ]:
data, labels_patients, indices_env, name_envs, eids = dataT.get_tree_data(with_env=False, load_possible=True, only_relevant=False)

In [ ]:
data_t, labels_patients_t= data[:10000], labels_patients[:10000]

In [ ]:
data_train, data_test, labels_train, labels_test = train_test_split(data_t, labels_patients_t, test_size = 1-prop_train_test, random_state=42)

In [ ]:
### maskage:
nb_features = data_test.shape[1]
indices_mask_train = np.random.randint(0, nb_features, len(data_train))
data_mask_train = data_train.copy()
data_mask_train[np.arange(len(data_mask_train)), indices_mask_train] = -1

indices_mask_test = np.random.randint(0, nb_features, len(data_test))
data_mask_test = data_test.copy()
data_mask_test[np.arange(len(data_mask_test)), indices_mask_test] = -1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

class Generator(nn.Module):
    def __init__(self, latent_dim, feature_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, feature_dim),
            nn.Sigmoid()  # Assuming input data range [0, 1]
        )

    def forward(self, data_mask, data_truth):
        data_gen = self.model(data_mask)


        return data_gen, torch.norm(data_gen - data_truth)/np.sqrt(len(data_gen))

    def eval(self, data_mask, data_truth):
        data_gen = self.model(data_mask)
        indices_mask = np.where(data)

class Discriminator(nn.Module):
    def __init__(self, feature_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(feature_dim, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, features):
        validity = self.model(features)
        return validity



In [ ]:
data_gen, loss = generator(data_mask_tensor_test, data_truth_tensor_test)
indices_mask = torch.where(data_mask_tensor_test== -1)
truth = data_truth_tensor_test[indices_mask]
gen = data_gen[indices_mask]


In [ ]:
torch.sum(truth==0)

In [ ]:
# Hyperparameters
latent_dim = nb_features
feature_dim = nb_features # Number of features in your input data
lr = 0.0002
batch_size = 64
epochs = 10

In [ ]:
generator = Generator(latent_dim, feature_dim)
discriminator = Discriminator(feature_dim)

# Loss function and optimizer
adversarial_loss = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

# Convert your data to PyTorch TensorDataset
data_mask_tensor_train = torch.tensor(data_mask_train, dtype=torch.float32)
data_truth_tensor_train = torch.tensor(data_train, dtype=torch.float32)


# Convert your data to PyTorch TensorDataset
data_mask_tensor_test = torch.tensor(data_mask_test, dtype=torch.float32)
data_truth_tensor_test = torch.tensor(data_test, dtype=torch.float32)


In [ ]:
dataloader = DataLoader(list(zip(data_mask_tensor, data_truth_tensor)), batch_size=20)

In [ ]:
d_gen, loss = generator(data_mask_tensor_test, data_truth_tensor_test)


In [ ]:
loss

In [ ]:

# Training Loop
for epoch in range(epochs):
    for i, (d_mask, d_truth) in enumerate(dataloader):
        
        # Train Generator
        optimizer_G.zero_grad()
        d_gen, loss = generator(d_mask, d_truth)
        loss.backward()
        optimizer_G.step()
        """
        # Train Discriminator
        optimizer_D.zero_grad()
        d_real_loss = adversarial_loss(discriminator(features), valid)
        d_fake_loss = adversarial_loss(discriminator(gen_features.detach()), fake)
        d_loss = (d_real_loss + d_fake_loss) / 2
        d_loss.backward()
        optimizer_D.step()
        """
        print(
            "[Epoch %d/%d] [Batch %d/%d] [G loss: %f]"
            % (epoch, epochs, i, len(dataloader), loss.item())
        )
